# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and validation data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert the scaled arrays back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns)


In [4]:
# Create a LinearRegression model
baseline_model = LinearRegression()

# Fit the model on the scaled training data
baseline_model.fit(X_train_scaled, y_train)

# Calculate a baseline r-squared score on the training data
baseline_r2 = baseline_model.score(X_train_scaled, y_train)
print(f"Baseline R-squared score on training data: {baseline_r2}")


Baseline R-squared score on training data: 0.7868344817421309


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure
interaction_scores = []

# Find combinations of columns and loop over them
for col1, col2 in combinations(X_train.columns, 2):
    # Make copies of X_train and X_val
    X_train_copy = X_train_scaled.copy()
    X_val_copy = X_val_scaled.copy()
    
    # Add interaction term to data
    interaction_term = X_train_copy[col1] * X_train_copy[col2]
    interaction_term_val = X_val_copy[col1] * X_val_copy[col2]
    interaction_name = f"{col1}_{col2}"
    X_train_copy[interaction_name] = interaction_term
    X_val_copy[interaction_name] = interaction_term_val
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    model = LinearRegression()
    model.fit(X_train_copy, y_train)
    r2_score = model.score(X_val_copy, y_val)
    
    # Append to data structure
    interaction_scores.append((interaction_name, r2_score))

# Sort and subset the data structure to find the top 7
interaction_scores.sort(key=lambda x: x[1], reverse=True)
top_7_interactions = interaction_scores[:7]

# Print the top 7 interactions
for interaction, score in top_7_interactions:
    print(f"Interaction: {interaction}, R-squared: {score}")


Interaction: LotArea_1stFlrSF, R-squared: 0.7211105666140573
Interaction: LotArea_TotalBsmtSF, R-squared: 0.7071649207050115
Interaction: LotArea_GrLivArea, R-squared: 0.6690980823779022
Interaction: LotArea_Fireplaces, R-squared: 0.6529699515652587
Interaction: 2ndFlrSF_TotRmsAbvGrd, R-squared: 0.6472994890405193
Interaction: OverallCond_TotalBsmtSF, R-squared: 0.6429019879233768
Interaction: OverallQual_2ndFlrSF, R-squared: 0.6422324294284368


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Loop over top 7 interactions
for interaction, score in top_7_interactions:
    # Extract column names from data structure
    col1, col2 = interaction.split('_')
    
    # Construct new column name
    interaction_name = f"{col1}_{col2}"
    
    # Add new column to X_train and X_val
    X_train_scaled[interaction_name] = X_train_scaled[col1] * X_train_scaled[col2]
    X_val_scaled[interaction_name] = X_val_scaled[col1] * X_val_scaled[col2]


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [7]:
# Set up data structure
polynomial_scores = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in [2, 3, 4]:
        # Make a copy of X_train and X_val
        X_train_copy = X_train_scaled.copy()
        X_val_copy = X_val_scaled.copy()
        
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        poly_train = poly.fit_transform(X_train[[col]])
        poly_val = poly.transform(X_val[[col]])
        
        # Convert the result to a DataFrame
        poly_train_df = pd.DataFrame(poly_train, columns=[f"{col}^{i}" for i in range(1, degree+1)])
        poly_val_df = pd.DataFrame(poly_val, columns=[f"{col}^{i}" for i in range(1, degree+1)])
        
        # Add polynomial to data
        X_train_copy = pd.concat([X_train_copy.drop(columns=[col]), poly_train_df], axis=1)
        X_val_copy = pd.concat([X_val_copy.drop(columns=[col]), poly_val_df], axis=1)
        
        # Find r-squared score on validation
        model = LinearRegression()
        model.fit(X_train_copy, y_train)
        r2_score = model.score(X_val_copy, y_val)
        
        # Append to data structure
        polynomial_scores.append((col, degree, r2_score))

# Sort and subset the data structure to find the top 7
polynomial_scores.sort(key=lambda x: x[2], reverse=True)
top_7_polynomials = polynomial_scores[:7]

# Print the top 7 polynomials
for col, degree, score in top_7_polynomials:
    print(f"Column: {col}, Degree: {degree}, R-squared: {score}")


Column: GrLivArea, Degree: 3, R-squared: 0.8283345026130584
Column: OverallQual, Degree: 3, R-squared: 0.8068023565969227
Column: OverallQual, Degree: 4, R-squared: 0.8033455378866836
Column: OverallQual, Degree: 2, R-squared: 0.802583337359408
Column: 2ndFlrSF, Degree: 3, R-squared: 0.7767514355074356
Column: 2ndFlrSF, Degree: 4, R-squared: 0.7696909328107031
Column: LotArea, Degree: 3, R-squared: 0.7647014750792784


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [8]:
# Your code here

# Filter out duplicates
unique_polynomials = {}
for col, degree, score in top_7_polynomials:
    if col not in unique_polynomials or unique_polynomials[col][1] < score:
        unique_polynomials[col] = (degree, score)

# Loop over remaining results
for col, (degree, score) in unique_polynomials.items():
    # Create polynomial terms
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_train = poly.fit_transform(X_train_scaled[[col]])
    poly_val = poly.transform(X_val_scaled[[col]])
    
    # Convert the result to a DataFrame
    poly_train_df = pd.DataFrame(poly_train, columns=[f"{col}^{i}" for i in range(1, degree+1)])
    poly_val_df = pd.DataFrame(poly_val, columns=[f"{col}^{i}" for i in range(1, degree+1)])
    
    # Concat new polynomials to X_train and X_val
    X_train_scaled = pd.concat([X_train_scaled.drop(columns=[col]), poly_train_df], axis=1)
    X_val_scaled = pd.concat([X_val_scaled.drop(columns=[col]), poly_val_df], axis=1)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [9]:
# Print the columns of the final datasets
print("Columns in X_train_scaled:")
print(X_train_scaled.columns)

print("\nColumns in X_val_scaled:")
print(X_val_scaled.columns)

Columns in X_train_scaled:
Index(['OverallCond', 'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'GarageArea',
       'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF',
       'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd',
       'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'GrLivArea^1',
       'GrLivArea^2', 'GrLivArea^3', 'OverallQual^1', 'OverallQual^2',
       'OverallQual^3', '2ndFlrSF^1', '2ndFlrSF^2', '2ndFlrSF^3', 'LotArea^1',
       'LotArea^2', 'LotArea^3'],
      dtype='object')

Columns in X_val_scaled:
Index(['OverallCond', 'TotalBsmtSF', '1stFlrSF', 'TotRmsAbvGrd', 'GarageArea',
       'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF',
       'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd',
       'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'GrLivArea^1',
       'GrLivArea^2', 'GrLivArea^3', 'OverallQual^1', 'OverallQual^2',
       'OverallQual^3', '2ndFlrSF^1', '2ndFlrSF^2', '2ndFlrSF^3', 'LotArea^1',
       'L

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [10]:
# Fit the model on the full training data with interactions and polynomial terms
full_model = LinearRegression()
full_model.fit(X_train_scaled, y_train)

# Calculate the R-squared score on the training data
train_r2 = full_model.score(X_train_scaled, y_train)
print(f"Full model R-squared score on training data: {train_r2}")

# Calculate the R-squared score on the validation data
val_r2 = full_model.score(X_val_scaled, y_val)
print(f"Full model R-squared score on validation data: {val_r2}")

Full model R-squared score on training data: 0.8514716747836066
Full model R-squared score on validation data: 0.795831492875674


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [11]:
# Define a range of n_features_to_select values
n_features_range = [5, 10, 15, 20, 25]

# Loop over the range of n_features_to_select values
for n_features in n_features_range:
    # Initialize RFE with the LinearRegression model and the current n_features_to_select
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features)
    
    # Fit RFE on the scaled training data
    rfe.fit(X_train_scaled, y_train)
    
    # Calculate the R-squared score on the training data
    train_r2_rfe = rfe.score(X_train_scaled, y_train)
    
    # Calculate the R-squared score on the validation data
    val_r2_rfe = rfe.score(X_val_scaled, y_val)
    
    # Print the results
    print(f"n_features_to_select: {n_features}")
    print(f"Train R-squared: {train_r2_rfe}")
    print(f"Validation R-squared: {val_r2_rfe}")
    print(f"Number of features selected: {n_features}")
    print("-" * 30)


n_features_to_select: 5
Train R-squared: 0.776039994126505
Validation R-squared: 0.6352981725272362
Number of features selected: 5
------------------------------
n_features_to_select: 10
Train R-squared: 0.8235682746440406
Validation R-squared: 0.7699939539204789
Number of features selected: 10
------------------------------
n_features_to_select: 15
Train R-squared: 0.8357048475147715
Validation R-squared: 0.8301279241242279
Number of features selected: 15
------------------------------
n_features_to_select: 20
Train R-squared: 0.8500601361669062
Validation R-squared: 0.7904975647411846
Number of features selected: 20
------------------------------
n_features_to_select: 25
Train R-squared: 0.8514716747836066
Validation R-squared: 0.795831492875674
Number of features selected: 25
------------------------------


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [12]:
# Define a range of alpha values
alpha_range = [0.01, 0.1, 1, 10, 100]

# Loop over the range of alpha values
for alpha in alpha_range:
    # Initialize Lasso with the current alpha value
    lasso = Lasso(alpha=alpha)
    
    # Fit Lasso on the scaled training data
    lasso.fit(X_train_scaled, y_train)
    
    # Calculate the R-squared score on the training data
    train_r2_lasso = lasso.score(X_train_scaled, y_train)
    
    # Calculate the R-squared score on the validation data
    val_r2_lasso = lasso.score(X_val_scaled, y_val)
    
    # Print the results
    print(f"Alpha: {alpha}")
    print(f"Train R-squared: {train_r2_lasso}")
    print(f"Validation R-squared: {val_r2_lasso}")
    print(f"Number of features used: {np.sum(lasso.coef_ != 0)}")
    print("-" * 30)


Alpha: 0.01
Train R-squared: 0.8514716704440692
Validation R-squared: 0.7958035206136165
Number of features used: 25
------------------------------
Alpha: 0.1
Train R-squared: 0.8514716692171405
Validation R-squared: 0.7958024299368864
Number of features used: 25
------------------------------
Alpha: 1
Train R-squared: 0.8514716337254029
Validation R-squared: 0.7957915028842499
Number of features used: 25
------------------------------
Alpha: 10
Train R-squared: 0.8514689564926174
Validation R-squared: 0.7956802495054591
Number of features used: 25
------------------------------
Alpha: 100
Train R-squared: 0.8512212885336649
Validation R-squared: 0.7964934924025951
Number of features used: 24
------------------------------


Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

In [13]:
"""
For RFE the model with the best validation R-Squared was using 20 features

For Lasso the model with the best validation R-Squared was using an alpha of 10000

The Lasso result was a bit better so let's go with that and the 12 features
that it selected
"""

"\nFor RFE the model with the best validation R-Squared was using 20 features\n\nFor Lasso the model with the best validation R-Squared was using an alpha of 10000\n\nThe Lasso result was a bit better so let's go with that and the 12 features\nthat it selected\n"

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Scale the test data
X_test_scaled = scaler.transform(X_test)

# Convert the scaled array back to DataFrame
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add the top 7 interaction terms to the test data
for interaction, score in top_7_interactions:
    col1, col2 = interaction.split('_')
    interaction_name = f"{col1}_{col2}"
    X_test_scaled[interaction_name] = X_test_scaled[col1] * X_test_scaled[col2]

# Add the top polynomial terms to the test data
for col, (degree, score) in unique_polynomials.items():
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_test = poly.fit_transform(X_test_scaled[[col]])
    poly_test_df = pd.DataFrame(poly_test, columns=[f"{col}^{i}" for i in range(1, degree+1)])
    X_test_scaled = pd.concat([X_test_scaled.drop(columns=[col]), poly_test_df], axis=1)

# Print the columns of the final test dataset to verify
print("Columns in X_test_scaled:")
print(X_test_scaled.columns)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [16]:
from sklearn.metrics import mean_squared_error

# Scale the test data
X_test_scaled = scaler.transform(X_test)

# Convert the scaled array back to DataFrame
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add the top 7 interaction terms to the test data
for interaction, score in top_7_interactions:
    col1, col2 = interaction.split('_')
    interaction_name = f"{col1}_{col2}"
    X_test_scaled[interaction_name] = X_test_scaled[col1] * X_test_scaled[col2]

# Add the top polynomial terms to the test data
for col, (degree, score) in unique_polynomials.items():
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_test = poly.fit_transform(X_test_scaled[[col]])
    poly_test_df = pd.DataFrame(poly_test, columns=[f"{col}^{i}" for i in range(1, degree+1)])
    X_test_scaled = pd.concat([X_test_scaled.drop(columns=[col]), poly_test_df], axis=1)

# Combine train and validation sets
X_train_val_scaled = pd.concat([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

# Fit the Lasso model on the combined train + validation set
lasso_final = Lasso(alpha=100)
lasso_final.fit(X_train_val_scaled, y_train_val)

# Calculate the R-squared score on the test data
test_r2 = lasso_final.score(X_test_scaled, y_test)
print(f"Test R-squared score: {test_r2}")

# Calculate the Mean Squared Error on the test data
test_mse = mean_squared_error(y_test, lasso_final.predict(X_test_scaled))
print(f"Test Mean Squared Error: {test_mse}")


Test R-squared score: 0.8516556180721855
Test Mean Squared Error: 1039198241.5826004


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.